# IMPORTS

In [163]:
import requests as re

from bs4 import BeautifulSoup

import pandas as pd


# ENVIROMENT VARIABLES

In [164]:
# Input data
game_title = 'warhammer'
platform = 'Windows'
currency = 'USD'

In [165]:
# Declaring platform dictionary
platform_dict_steam = {
    'Windows': 'win',
    'MacOS': 'mac',
    'Linux': 'linux',
}

In [166]:
platform_dict_gog = {
    'Windows': 'windows',
    'MacOS': 'mac',
    'Linux': 'linux',
}

In [167]:
country_dict_gog = {
    'USD': 'US',
    'EUR': 'ES'
}

In [168]:
platform_dict_cdkeys = {
    'Windows': 'Steam',
    'Xbox Series X/S': 'Xbox'
}

In [169]:
# Instant Gamin platfor ditioanry
platform_dict_instant_gaming = {
    'Windows': '1',
    'MacOS': '2',
    'Linux': '3',
    'Switch': '4',
    'Playstation 4': '5',
    'Playstation 5': '6',
    'Xbox One': '7',
    'Xbox Series X/S': '8',
    'GeForce Now': '9',
}

In [170]:
# Declaring currency dictionary
currency_dict_steam = {
    'USD': 'us',
    'EUR': 'es'
}

In [171]:
currency_dict_epic_games = {
    'USD': 'US',
    'EUR': 'ES'
}

# BUILDING DATA FRAMES

## STEAM

In [172]:
# Transforming platform and currency to steam format
cleared_platform = platform_dict_steam[platform]
cleared_currency = currency_dict_steam[currency]

In [173]:
# Building steam's url
steam_url = f'https://store.steampowered.com/search/?&term={game_title}&os={cleared_platform}&cc={cleared_currency}'
# 'https://store.steampowered.com/search/?term=monster+hunter&os=win&supportedlang=english&ndl=1'

In [174]:
# Getting steam's page
steam_object = re.get(steam_url)

In [175]:
# Chcecking if the page was successfully downloaded
steam_object.status_code

200

In [176]:
# Parsing the page
soup = BeautifulSoup(steam_object.content, 'html.parser')

In [177]:
# Get the titles list
titles = []
for x in soup.find_all('span', class_='title'):
    titles.append(x.text)

In [178]:
# Search for the 'Uninitialized' title and remove it
uninitialized = []
for i in range(0, len(titles) - 1):
    if titles[i] == 'Uninitialized':
        uninitialized.append(i)
        del titles[i]

In [179]:
# Get the prices list
prices = []
for x in soup.find_all('div', class_='col search_price_discount_combined responsive_secondrow'):
    try:
        prices.append(str(x).split('discount_final_price">')[1].split("</div>")[0].replace('$', '').replace('€', ''))
    except:
        prices.append('')

In [180]:
# Get the links list
links = []
for x in soup.find_all('a', class_='search_result_row ds_collapse_flag'):
    links.append(x.get('href'))


In [181]:
for link in links:
    if links.index(link) in uninitialized:
        links.remove(link)

In [182]:
#Get images
images = []
for x in soup.find_all('div', class_='col search_capsule'):
    images.append(str(x).split('src="')[1].split('"')[0])

In [183]:
for image in images:
    if images.index(image) in uninitialized:
        images.remove(image)

In [184]:
data = {
    'title': titles,
    'price': prices,
    'link': links,
    'image': images
}

In [185]:
print(f'{len(titles), len(prices), len(links), len(images)}')

(50, 50, 50, 50)


In [186]:
df_steam  = pd.DataFrame(data)

In [187]:
df_steam['vendor'] = 'Steam'

In [188]:
df_steam.head()

,title,price,link,image,vendor
0,"Warhammer 40,000: Darktide",19.99,https://store.steampowered.com/app/1361210/War...,https://shared.cloudflare.steamstatic.com/stor...,Steam
1,Total War: WARHAMMER III,59.99,https://store.steampowered.com/app/1142710/Tot...,https://shared.cloudflare.steamstatic.com/stor...,Steam
2,"Warhammer 40,000: Space Marine 2",59.99,https://store.steampowered.com/app/2183900/War...,https://shared.cloudflare.steamstatic.com/stor...,Steam
3,"Warhammer 40,000: Rogue Trader",49.99,https://store.steampowered.com/app/2186680/War...,https://shared.cloudflare.steamstatic.com/stor...,Steam
4,Total War: WARHAMMER II,59.99,https://store.steampowered.com/app/594570/Tota...,https://shared.cloudflare.steamstatic.com/stor...,Steam


## GOG

In [189]:
gog_url = f'https://catalog.gog.com/v1/catalog?countryCode={country_dict_gog[currency]}'
gog_params = {
    'limit': '48',
    'query': game_title,
    'system': platform_dict_gog[platform],
    'currencyCode': currency 
}


In [190]:
gog_params

{'limit': '48',
 'query': 'warhammer',
 'system': 'windows',
 'currencyCode': 'USD'}

In [191]:
gog_object = re.get(url=gog_url, params=gog_params)

In [192]:
gog_object.status_code

200

In [193]:
# Get titles, prices, links and images
titles = []
prices = []
links =  []
images = []
for product in gog_object.json()['products']:
    titles.append(product['title']) 
    prices.append(product['price']['final'])
    links.append(product['storeLink'])
    images.append(product['coverHorizontal'])


In [194]:
prices

['$0.89',
 '$2.49',
 '$1.49',
 '$8.99',
 '$3.99',
 '$14.94',
 '$5.24',
 '$8.99',
 '$2.49',
 '$12.74',
 '$11.24',
 '$5.94',
 '$8.99',
 '$5.94',
 '$9.74',
 '$2.99',
 '$8.99',
 '$2.49',
 '$8.99',
 '$2.99',
 '$8.99',
 '$3.49',
 '$8.99',
 '$3.99',
 '$1.49',
 '$2.69',
 '$2.49',
 '$13.49',
 '$17.99',
 '$3.89',
 '$11.99',
 '$49.99',
 '$14.99',
 '$29.99',
 '$4.99',
 '$64.98',
 '$99.96',
 '$3.49',
 '$10.49',
 '$13.19',
 '$10.49',
 '$9.99',
 '$8.99',
 '$3.49',
 '$1.39',
 '$1.39',
 '$23.99',
 '$19.99']

In [195]:
data  = {
    'title': titles,
    'price': prices,
    'link': links,
    'image': images
}

In [196]:
df_gog = pd.DataFrame(data)

In [197]:
df_gog['vendor'] = 'GOG'

In [198]:
df_gog.head()

,title,price,link,image,vendor
0,"Warhammer 40,000: Fire Warrior",$0.89,https://www.gog.com/en/game/warhammer_40000_fi...,https://images.gog-statics.com/aee5c75ab5ade63...,GOG
1,Warhammer: Mark of Chaos - Gold Edition,$2.49,https://www.gog.com/en/game/warhammer_mark_of_...,https://images.gog-statics.com/0059c74092f695a...,GOG
2,Warhammer: Shadow of the Horned Rat,$1.49,https://www.gog.com/en/game/warhammer_shadow_o...,https://images.gog-statics.com/1618a978f7a8844...,GOG
3,"Warhammer 40,000: Gladius - Ultima Founding",$8.99,https://www.gog.com/en/game/warhammer_40000_gl...,https://images.gog-statics.com/75f7aa9d069ebde...,GOG
4,Warhammer: Chaosbane Slayer Edition,$3.99,https://www.gog.com/en/game/warhammer_chaosban...,https://images.gog-statics.com/674c50f72e5aef2...,GOG


## CDKEYS

In [199]:
# Run the API in postman

In [200]:
cdkeys_url = 'https://muvyib7tey-dsn.algolia.net/1/indexes/*/queries?x-algolia-agent=Algolia%20for%20JavaScript%20(4.13.1)%3B%20Browser%3B%20instantsearch.js%20(4.41.0)%3B%20Magento2%20integration%20(3.10.5)%3B%20JS%20Helper%20(3.8.2'

In [201]:
cdkeys_form_data = {
    "requests": [
        {
            "indexName": "magento2_default_products",
            "params": f"highlightPreTag=__ais-highlight__&highlightPostTag=__%2Fais-highlight__&page=0&ruleContexts=%5B%22magento_filters%22%5D&hitsPerPage=24&clickAnalytics=true&query={game_title}&maxValuesPerFacet=10&facets=%5B%22restricted_countries.es_es%22%2C%22platforms.es_es%22%2C%22region.es_es%22%2C%22language.es_es%22%2C%22genres.es_es%22%2C%22price.EUR.default%22%5D&tagFilters=&facetFilters=%5B%22restricted_countries.es_es%3A-ES%22%2C%5B%22platforms.es_es%3ASteam%22%5D%5D&numericFilters=%5B%22visibility_search.default%3D1%22%2C%5B%22region_id.default%3D39%22%2C%22region_id.default%3D38%22%2C%22region_id.default%3D394%22%2C%22region_id.default%3D479%22%2C%22region_id.default%3D3382%22%2C%22region_id.default%3D3505%22%5D%5D"
        }
    ]
}

In [202]:
cdkeys_headers = {
    'Accept-Language':'es-ES;q=0.8,es;q=0.7',
    'Content-Type':'application/x-www-form-urlencoded',
    'Content-Length':'1472',
    'Host':'muvyib7tey-dsn.algolia.net',
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
    'X-Algolia-Api-Key':'ODNjY2VjZjExZGE2NTg3ZDkyMGQ4MjljYzYwM2U0NmRjYWI4MDgwNTQ0NjgzNmE2ZGQyY2ZmMDlkMzAyYTI4NXRhZ0ZpbHRlcnM9',
    'X-Algolia-Application-Id':'MUVYIB7TEY'
}

In [203]:
cdkeys_object = re.post(url=cdkeys_url, json=cdkeys_form_data, headers=cdkeys_headers)

In [204]:
# Get titles, prices, links and images
titles = []
prices = []
links =  []
images = []
for product in cdkeys_object.json()['results'][0]['hits']:
    titles.append(product['name']['default']) 
    prices.append(product['price'][currency]['default'])
    links.append(product['url']['default'])
    images.append(product['thumbnail_url']['default'])


In [205]:
data  = {
    'title': titles,
    'price': prices,
    'link': links,
    'image': images
}

In [206]:
df_cdkeys = pd.DataFrame(data)

In [207]:
df_cdkeys['vendor'] = 'CDKeys'

In [208]:
df_cdkeys.head()

,title,price,link,image,vendor
0,"Warhammer 40,000: Space Marine 2 PC",38.69,https://www.cdkeys.com/warhammer-40-000-space-...,https://cdn.cdkeys.com/140x140/media/catalog/p...,CDKeys
1,Total War: WARHAMMER III PC (WW),23.69,https://www.cdkeys.com/total-war-warhammer-iii...,https://cdn.cdkeys.com/140x140/media/catalog/p...,CDKeys
2,Total War: WARHAMMER Trilogy PC (WW),27.49,https://www.cdkeys.com/total-war-warhammer-tri...,https://cdn.cdkeys.com/140x140/media/catalog/p...,CDKeys
3,"Warhammer 40,000: Darktide PC",12.49,https://www.cdkeys.com/warhammer-40-000-darkti...,https://cdn.cdkeys.com/140x140/media/catalog/p...,CDKeys
4,"Warhammer 40,000: Rogue Trader PC",15.39,https://www.cdkeys.com/warhammer-40-000-rogue-...,https://cdn.cdkeys.com/140x140/media/catalog/p...,CDKeys


## INSTANT GAMING

In [209]:
url = f'https://www.instant-gaming.com/es/busquedas/?platform%5B%5D={platform_dict_instant_gaming[platform]}&query={game_title}&currency={currency}'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'
}

instant_gaming_object = re.get(url, headers=headers)

In [210]:
url

'https://www.instant-gaming.com/es/busquedas/?platform%5B%5D=1&query=warhammer&currency=USD'

In [211]:
instant_gaming_object.status_code

200

In [212]:
soup = BeautifulSoup(instant_gaming_object.content, 'html.parser')

In [213]:
# Get the titles list
titles = []
prices = []
images = []
links = []
for x in soup.find_all('div', class_='item force-badge'):
    titles.append(x.find('span', class_='title').text)
    try:
        prices.append(x.find('div', class_='price').text)
    except:
        prices.append('')
    images.append(str(x.find('img', class_='picture')).strip().split('src="')[1].split('"')[0])
    links.append(x.find('a').get('href'))

In [214]:
data  = {
    'title': titles,
    'price': prices,
    'link': links,
    'image': images
}

In [215]:
df_instant_gaming = pd.DataFrame(data)

In [216]:
df_instant_gaming['vendor'] = 'Instant-Gaming'

In [217]:
df_instant_gaming.head()

,title,price,link,image,vendor
0,"Warhammer 40,000: Mechanicus II",,https://www.instant-gaming.com/es/16897-compra...,https://gaming-cdn.com/images/products/16897/3...,Instant-Gaming
1,"Warhammer 40,000: Space Marine 2",$39.36,https://www.instant-gaming.com/es/10140-compra...,https://gaming-cdn.com/images/products/10140/3...,Instant-Gaming
2,"Warhammer 40,000: Space Marine 2 - Ultra Edition",$63.05,https://www.instant-gaming.com/es/16908-compra...,https://gaming-cdn.com/images/products/16908/3...,Instant-Gaming
3,"Warhammer 40,000: Space Marine 2 - Gold Edition",$56.69,https://www.instant-gaming.com/es/16909-compra...,https://gaming-cdn.com/images/products/16909/3...,Instant-Gaming
4,"Warhammer 40,000: Space Marine 2 - Season Pass",$31.07,https://www.instant-gaming.com/es/17572-compra...,https://gaming-cdn.com/images/products/17572/3...,Instant-Gaming


## EPIC GAMES

In [218]:
epic_games_url = f'https://epic-games-store.p.rapidapi.com/search/{game_title}/page/1/country/{currency_dict_steam[currency]}/locale/us'
epic_games_headers = {
	"x-rapidapi-key": "c6a5a891ecmsh03d9ec66402dd34p18a3c6jsnbe65087e49e4",
	"x-rapidapi-host": "epic-games-store.p.rapidapi.com"
}

In [219]:
epic_games_object = re.get(url=epic_games_url, headers=epic_games_headers)

In [220]:
epic_games_object

<Response [200]>

In [221]:
# Get titles, prices, links and images
titles = []
prices = []
links =  []
images = []
for product in epic_games_object.json()['Catalog']['searchStore']['elements']:
    titles.append(product['title']) 
    prices.append(product['price']['totalPrice']['discountPrice'] / 100)
    term = product['title'].replace(' ', '+')
    links.append(f'https://store.epicgames.com/en-US/browse?q={term}&sortBy=relevancy&sortDir=DESC&count=40')
    images.append('Not available')


In [222]:
data  = {
    'title': titles,
    'price': prices,
    'link': links,
    'image': images
}

In [223]:
df_epic_games = pd.DataFrame(data)

In [224]:
df_epic_games['vendor'] = 'Epic Games'

In [225]:
df_epic_games.head()

,title,price,link,image,vendor
0,"Warhammer 40,000: Rogue Trader",49.99,https://store.epicgames.com/en-US/browse?q=War...,Not available,Epic Games
1,"Warhammer 40,000: Gladius - Relics of War",3.99,https://store.epicgames.com/en-US/browse?q=War...,Not available,Epic Games
2,Total War: WARHAMMER III - Forge of the Chaos ...,24.99,https://store.epicgames.com/en-US/browse?q=Tot...,Not available,Epic Games
3,"Warhammer 40,000: Battlesector",19.99,https://store.epicgames.com/en-US/browse?q=War...,Not available,Epic Games
4,"Warhammer 40,000: Boltgun",21.99,https://store.epicgames.com/en-US/browse?q=War...,Not available,Epic Games


# BUILD THE FINAL DATAFRAME

In [226]:
df_final = pd.concat([df_steam, df_gog, df_cdkeys, df_instant_gaming, df_epic_games]).reset_index(drop=True)

In [227]:
df_final

,title,price,link,image,vendor
0,"Warhammer 40,000: Darktide",19.99,https://store.steampowered.com/app/1361210/War...,https://shared.cloudflare.steamstatic.com/stor...,Steam
1,Total War: WARHAMMER III,59.99,https://store.steampowered.com/app/1142710/Tot...,https://shared.cloudflare.steamstatic.com/stor...,Steam
2,"Warhammer 40,000: Space Marine 2",59.99,https://store.steampowered.com/app/2183900/War...,https://shared.cloudflare.steamstatic.com/stor...,Steam
3,"Warhammer 40,000: Rogue Trader",49.99,https://store.steampowered.com/app/2186680/War...,https://shared.cloudflare.steamstatic.com/stor...,Steam
4,Total War: WARHAMMER II,59.99,https://store.steampowered.com/app/594570/Tota...,https://shared.cloudflare.steamstatic.com/stor...,Steam
...,...,...,...,...,...
217,Total War: WARHAMMER III - Shadows of Change,24.99,https://store.epicgames.com/en-US/browse?q=Tot...,Not available,Epic Games
218,"Warhammer 40,000: Boltgun - Forges of Corrupti...",5.99,https://store.epicgames.com/en-US/browse?q=War...,Not available,Epic Games
219,Total War: WARHAMMER - Grombrindal The White D...,0.0,https://store.epicgames.com/en-US/browse?q=Tot...,Not available,Epic Games
220,Warhammer Age of Sigmar: Storm Ground,19.99,https://store.epicgames.com/en-US/browse?q=War...,Not available,Epic Games


In [228]:
df_final['price'] = pd.to_numeric(df_final['price'].astype('str').str.replace('$','').replace('Free', '').fillna(0))

In [229]:
df_final.sort_values('price')

,title,price,link,image,vendor
219,Total War: WARHAMMER - Grombrindal The White D...,0.0,https://store.epicgames.com/en-US/browse?q=Tot...,Not available,Epic Games
206,Total War: WARHAMMER - Assembly Kit,0.0,https://store.epicgames.com/en-US/browse?q=Tot...,Not available,Epic Games
203,Total War: WARHAMMER II - Mortal Empires,0.0,https://store.epicgames.com/en-US/browse?q=Tot...,Not available,Epic Games
210,Total War: WARHAMMER II – Assembly Kit,0.0,https://store.epicgames.com/en-US/browse?q=Tot...,Not available,Epic Games
213,Total War: WARHAMMER II - Steps of Isha,0.0,https://store.epicgames.com/en-US/browse?q=Tot...,Not available,Epic Games
...,...,...,...,...,...
43,Warhammer The Horus Heresy: Legions,NaN,https://store.steampowered.com/app/1031140/War...,https://shared.cloudflare.steamstatic.com/stor...,Steam
122,"Warhammer 40,000: Mechanicus II",NaN,https://www.instant-gaming.com/es/16897-compra...,https://gaming-cdn.com/images/products/16897/3...,Instant-Gaming
134,Warhammer: Vermintide 2 - Collector's Edition,NaN,https://www.instant-gaming.com/es/2754-comprar...,https://gaming-cdn.com/images/products/2754/38...,Instant-Gaming
169,"Warhammer 40,000: Gladius - Relics of War",NaN,https://www.instant-gaming.com/es/5117-comprar...,https://gaming-cdn.com/images/products/5117/38...,Instant-Gaming


In [64]:
df_final.to_csv("sample_data.csv")

End of the line